In [ ]:
''' 
Classificando as melhores empresas, estou multiplicando os valores da analise de correlação pelo valor
da empresa naquele atributo, não estou usando valores negativos da analise, o primeiro resultado foi
semelhante a esse, porém trabalhando com muitos dados em falta. Nessa análise, empresas com atributos
em falta estão sendo descartadas para classificação'''

In [1]:
#Importando bibliotecas e abrindo arquivos necessários
import pandas as pd
import numpy as np
from Análise_de_Correlação import analiseCorr
from Normalização import normalizaDf
df= pd.read_excel("input_acoes_NotasFundamentei_20200517.xlsx")
abas= pd.ExcelFile("input_planilhaCanalHolder_20200211.xlsx")
MA=abas.parse('Múltiplos Ações')
FA=abas.parse('Fundamentos Ações')

Lucro Líquido                             0.300
Fluxo de Caixa de Financiamentos (FCF)    0.254
Crescimento_Médio_Anual                   0.244
Cotação                                   0.238
2015                                      0.236
EV/EBITDA                                 0.220
ROA                                       0.219
P/VPA                                     0.198
PSR                                       0.176
ROE                                       0.174
Valor de Mercado                          0.123
Valor_de_Mercado                          0.123
Fluxo de Caixa de Investimentos (FCI)     0.118
Participação no Ibovespa                  0.094
Participação_no_Ibovespa                  0.094
Participação no IBRX 100                  0.094
Participação_no_IBRX_100                  0.094
Capex/FCO                                 0.054
Volume_Negociado                          0.053
Volume Negociado                          0.053
Lucro Operacional (EBIT)                

In [2]:
#Fazendo alterações no arquivo, removendo espaços nos nomes das colunas
MA.columns = [str(c).replace(' ', '_') for c in MA.columns]
FA.columns = [str(c).replace(' ', '_') for c in FA.columns]
df.columns = [str(c).replace(' ', '_') for c in df.columns]

In [3]:
#Join das tabelas "Fundamentos ações" e "NotasCanalFundamentei" e um merge entre as duas primeiras abas do arquivo input_planilhaCanalHolder_20200211.xlsx
MA=MA.join(df.set_index("Código_de_Neg."),on="Código_de_Neg.")
uniaoAbas = pd.merge(MA, FA, on="Nome", how='outer', suffixes=('', '_y'))
uniaoAbas = uniaoAbas.drop(list(uniaoAbas.filter(regex='_y$')), axis=1)
#Remoção de uma coluna inútil
uniaoAbas.drop("Unnamed:_0",axis=1,inplace=True)

In [55]:
teste=uniaoAbas.replace("-",np.nan)

In [56]:
#DataFrame inicial porém sem empresas com dados faltando
testeExclusao=teste.dropna()

In [57]:
dfNormalizado=normalizaDf(testeExclusao)

In [58]:
#Chamando a função principal
analiseCorrelação=analiseCorr("NotaFundamentei20200201",dfNormalizado)[2:]

In [59]:
def calculoAutomatizadoScore(analise,dFrame):
    #print(dFrame)
    Score=[]
    for linha in dFrame.axes[0]:
        count=0
        score=0
        for peso in analise.values:
            if peso>0:
                score+=dFrame[analise.axes[0][count]][linha]*peso
                count+=1
        #print(score)
        Score.append(score)
    dFrame["Score"]=Score
    return dFrame
""" Paramêtros:
        analise: Analise de Correlação feita
        dataFrame: DataFrame contendo as empresas a serem rankeadas
    Return: O mesmo DataFrame do parâmetro, agora contendo a coluna Score """

' Paramêtros:\n        analise: Analise de Correlação feita\n        dataFrame: DataFrame contendo as empresas a serem rankeadas\n    Return: O mesmo DataFrame do parâmetro, agora contendo a coluna Score '

In [60]:
testeAutomatizado=calculoAutomatizadoScore(analiseCorrelação,dfNormalizado)

In [61]:
testeAutomatizado["Nome"]=testeExclusao["Nome"]

In [62]:
testeAutomatizado.sort_values("Score",ascending=False)[["Nome","Score"]]

,Nome,Score
232,Magaz Luiza,1.548887
31,Ambev S/A,1.463795
39,B3,1.341267
278,RaiaDrogasil,1.263581
385,Weg,1.228396
218,Localiza,1.193311
228,Lojas Renner,1.192632
322,Smiles,1.192024
136,Engie Brasil,1.175926
356,Totvs,1.127706


In [45]:
#Posição da empresa com maior Score na tabela de melhores empresas, usada como paramêtro
df[df["Código_de_Neg."]=="MGLU3"]

,Código_de_Neg.,NotaFundamentei,NotaFundamentei20200201
20,MGLU3,4.1,4.1
